<a href="https://colab.research.google.com/github/JkoolCode/TFM_Maestria_IA/blob/main/00_Extraccion_de_datos_y_Preprocesamiento_Options_SPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install py7zr

# Librerias

In [ ]:
import py7zr
import pandas as pd
import os
import numpy as np

# Leer Dataset

In [ ]:
anio = '2020'

In [ ]:
# 2020 y 2021
if (anio == '2020') or (anio == '2021'):
  # Ruta del archivo descargado
  extract_path = "spy_eod_" + anio

  for i in range(1, 2):
    # Extraer el contenido a una carpeta temporal
    archive_path = f"spy_eod_{anio}.7z"
    with py7zr.SevenZipFile(archive_path, mode='r') as archive:
        archive.extractall(path=extract_path)

In [ ]:
# 2022 y 2023
if (anio == '2022') or (anio == '2023'):
  # Ruta del archivo descargado
  extract_path = "spy_eod_" + anio

  for i in range(1, 5):
    # Extraer el contenido a una carpeta temporal
    archive_path = f"spy_eod_{anio}q{i}.7z"
    with py7zr.SevenZipFile(archive_path, mode='r') as archive:
        archive.extractall(path=extract_path)

In [ ]:
len(os.listdir(extract_path))

12

In [ ]:
# Revisar qué archivo se extrajo
print(os.listdir(extract_path))

['spy_eod_202304.txt', 'spy_eod_202308.txt', 'spy_eod_202306.txt', 'spy_eod_202310.txt', 'spy_eod_202301.txt', 'spy_eod_202307.txt', 'spy_eod_202309.txt', 'spy_eod_202311.txt', 'spy_eod_202302.txt', 'spy_eod_202303.txt', 'spy_eod_202305.txt', 'spy_eod_202312.txt']


In [ ]:
# Leer todos los .txt como DataFrames
dataframes = []
for f in os.listdir(extract_path):
    if f.endswith('.txt'):
        file_path = os.path.join(extract_path, f)
        print(f"Leyendo {file_path} ...")
        df = pd.read_csv(file_path, sep=',')  # Usa sep=',' explícitamente
        dataframes.append(df)

# Combinar todos los DataFrames en uno solo (si tienen mismas columnas)
df_all = pd.concat(dataframes, ignore_index=True)

Leyendo spy_eod_2023/spy_eod_202304.txt ...
Leyendo spy_eod_2023/spy_eod_202308.txt ...


/tmp/ipython-input-57713689.py:7: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=',')  # Usa sep=',' explícitamente


Leyendo spy_eod_2023/spy_eod_202306.txt ...
Leyendo spy_eod_2023/spy_eod_202310.txt ...
Leyendo spy_eod_2023/spy_eod_202301.txt ...
Leyendo spy_eod_2023/spy_eod_202307.txt ...
Leyendo spy_eod_2023/spy_eod_202309.txt ...
Leyendo spy_eod_2023/spy_eod_202311.txt ...


/tmp/ipython-input-57713689.py:7: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=',')  # Usa sep=',' explícitamente


Leyendo spy_eod_2023/spy_eod_202302.txt ...
Leyendo spy_eod_2023/spy_eod_202303.txt ...
Leyendo spy_eod_2023/spy_eod_202305.txt ...


/tmp/ipython-input-57713689.py:7: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=',')  # Usa sep=',' explícitamente


Leyendo spy_eod_2023/spy_eod_202312.txt ...


In [ ]:
df_all.shape

(972162, 33)

# Data wrangling

## Formato columnas

In [ ]:
def clean_columns(df):
  df.columns = df.columns.str.lstrip().str.replace('[', '', regex=False).str.replace(']', '', regex=False).str.lower()
  return df

In [ ]:
#eliminar espacio en blanco, corchetes y aplicar minusculas en los nombres de las columnas
df_all = clean_columns(df_all)

In [ ]:
df_all.columns

Index(['quote_unixtime', 'quote_readtime', 'quote_date', 'quote_time_hours',
       'underlying_last', 'expire_date', 'expire_unix', 'dte', 'c_delta',
       'c_gamma', 'c_vega', 'c_theta', 'c_rho', 'c_iv', 'c_volume', 'c_last',
       'c_size', 'c_bid', 'c_ask', 'strike', 'p_bid', 'p_ask', 'p_size',
       'p_last', 'p_delta', 'p_gamma', 'p_vega', 'p_theta', 'p_rho', 'p_iv',
       'p_volume', 'strike_distance', 'strike_distance_pct'],
      dtype='object')

In [ ]:
important_cols = ['quote_date', 'expire_date', 'dte', 'underlying_last', 'strike', 'strike_distance', 'strike_distance_pct',
                  'c_last', 'c_bid', 'c_ask', 'c_iv', 'c_volume', 'c_delta', 'c_gamma', 'c_vega', 'c_theta']
df_all = df_all[important_cols]

In [ ]:
df_all.dtypes

,0
quote_date,object
expire_date,object
dte,float64
underlying_last,float64
strike,float64
strike_distance,float64
strike_distance_pct,float64
c_last,float64
c_bid,float64
c_ask,float64


## Revision de nulos

In [ ]:
df_all = df_all.replace('', np.nan)
df_all = df_all.replace(' ', np.nan)
df_all.isnull().sum()

,0
quote_date,0
expire_date,0
dte,0
underlying_last,0
strike,0
strike_distance,0
strike_distance_pct,0
c_last,0
c_bid,0
c_ask,0


In [ ]:
# c_volume_mode = df_all['c_volume'].mode()
# df_all['c_volume'] = df_all['c_volume'].replace(np.nan, c_volume_mode[0])

In [ ]:
df_all.isnull().sum()

,0
quote_date,0
expire_date,0
dte,0
underlying_last,0
strike,0
strike_distance,0
strike_distance_pct,0
c_last,0
c_bid,0
c_ask,0


In [ ]:
# Eliminar nulos
df_all = df_all.dropna().reset_index()
df_all = df_all.drop(columns=['index'])

In [ ]:
df_all.isnull().sum()

,0
quote_date,0
expire_date,0
dte,0
underlying_last,0
strike,0
strike_distance,0
strike_distance_pct,0
c_last,0
c_bid,0
c_ask,0


In [ ]:
# Dar formatos a tipo de datos
df_all['quote_date'] = pd.to_datetime(df_all['quote_date'])
df_all['expire_date'] = pd.to_datetime(df_all['expire_date'])
df_all['c_iv'] = df_all['c_iv'].astype(float)
df_all['c_volume'] = df_all['c_volume'].astype(float)

df_all['c_last'] = df_all['c_last'].astype(float)
df_all['c_bid'] = df_all['c_bid'].astype(float)
df_all['c_ask'] = df_all['c_ask'].astype(float)
df_all['c_delta'] = df_all['c_delta'].astype(float)
df_all['c_gamma'] = df_all['c_gamma'].astype(float)
df_all['c_vega'] = df_all['c_vega'].astype(float)
df_all['c_theta'] = df_all['c_theta'].astype(float)

In [ ]:
pd.to_datetime('2025-11-07') - pd.to_datetime('2025-11-07')

Timedelta('0 days 00:00:00')

In [ ]:
df_all.dtypes

,0
quote_date,datetime64[ns]
expire_date,datetime64[ns]
dte,float64
underlying_last,float64
strike,float64
strike_distance,float64
strike_distance_pct,float64
c_last,float64
c_bid,float64
c_ask,float64


## Variables calculadas

In [ ]:
# Variables calculados
df_all['daysToExp'] = (df_all['expire_date'] - df_all['quote_date']).dt.days
df_all['c_mid_price'] = (df_all['c_bid'] + df_all['c_ask']) / 2
df_all['moneyness_category'] = np.where(
    df_all['underlying_last'] > df_all['strike'], 'ITM',
    np.where(df_all['underlying_last'] < df_all['strike'], 'OTM', 'ATM')
)

In [ ]:
df_all.head()

,quote_date,expire_date,dte,underlying_last,strike,strike_distance,strike_distance_pct,c_last,c_bid,c_ask,c_iv,c_volume,c_delta,c_gamma,c_vega,c_theta,daysToExp,c_mid_price,moneyness_category
0,2023-04-03,2023-04-03,0.0,410.93,320.0,90.9,0.221,90.46,90.88,91.01,0.00021,3.0,1.00000,0.00000,0.00000,-0.03217,0,90.945,ITM
1,2023-04-03,2023-04-03,0.0,410.93,330.0,80.9,0.197,79.23,80.88,81.02,0.00004,3.0,1.00000,0.00000,0.00000,-0.03254,0,80.950,ITM
2,2023-04-03,2023-04-03,0.0,410.93,334.0,76.9,0.187,64.42,76.88,77.02,-0.00021,0.0,1.00000,0.00000,0.00000,-0.03375,0,76.950,ITM
3,2023-04-03,2023-04-03,0.0,410.93,388.0,22.9,0.056,21.49,22.66,23.55,0.67965,208.0,0.97155,0.00517,0.01180,-0.17506,0,23.105,ITM
4,2023-04-03,2023-04-03,0.0,410.93,390.0,20.9,0.051,21.08,20.43,21.55,0.48282,31.0,0.99224,0.00227,0.00375,-0.06086,0,20.990,ITM


## Funciones

In [ ]:
def get_fridays_in_range(start_date, end_date):
  """
  Returns a list of Fridays within a given date range.

  Args:
    start_date: The start date of the range (inclusive).
    end_date: The end date of the range (inclusive).

  Returns:
    A list of datetime objects representing the Fridays in the range.
  """
  date_range = pd.date_range(start=start_date, end=end_date, freq='D')
  fridays = [date for date in date_range if date.dayofweek == 4] # Monday=0, Sunday=6
  return fridays

In [ ]:
# def filter_by_closest_friday_expiry(df, fridays_list):
#     """
#     Filters the DataFrame to include rows where expire_date is between quote_date
#     and the closest Friday in fridays_list that is within 0 to 5 days after quote_date.

#     Args:
#         df: The input DataFrame with 'quote_date' and 'expire_date' columns (datetime objects).
#         fridays_list: A list of datetime objects representing Fridays.

#     Returns:
#         A filtered DataFrame.
#     """
#     filtered_df_list = []
#     for quote_date in df['quote_date'].unique():
#         # Find the closest Friday in fridays_list that is >= quote_date and within 5 days
#         closest_friday = None
#         for friday in fridays_list:
#             if friday >= quote_date and (friday - quote_date).days <= 5:
#                 if closest_friday is None or friday < closest_friday:
#                     closest_friday = friday
#                     break

#         if closest_friday:
#             # Filter for the current quote_date and the identified expiry date range
#             temp_df = df[
#                 (df['quote_date'] == quote_date) &
#                 (df['expire_date'] >= quote_date) &
#                 (df['expire_date'] <= closest_friday)
#             ].copy()
#             filtered_df_list.append(temp_df)

#     if filtered_df_list:
#         return pd.concat(filtered_df_list, ignore_index=True)
#     else:
#         return pd.DataFrame(columns=df.columns) # Return empty DataFrame if no matches

In [ ]:
def filter_by_closest_friday_expiry(df, fridays_list):
    """
    Filters the DataFrame to include rows where expire_date is between quote_date
    and the closest Friday in fridays_list that is within 0 to 5 days after quote_date.

    Args:
        df: The input DataFrame with 'quote_date' and 'expire_date' columns (datetime objects).
        fridays_list: A list of datetime objects representing Fridays.

    Returns:
        A filtered DataFrame.
    """
    filtered_df_list = []
    for quote_date in df['quote_date'].unique():
        # print("quote_date: ", quote_date)
        closest_friday = None
        #for friday in fridays_list:
        for i in range(len(fridays_list)):
          friday = fridays_list[i]
          days_diff = (friday - quote_date).days
          if (friday >= quote_date) and (days_diff >= 2) and (days_diff <= 4):
            # print("days_diff", days_diff)
            if closest_friday is None or friday < closest_friday:
                closest_friday = friday
                break
          elif (friday >= quote_date) and (days_diff <= 1):
            # print("days_diff", days_diff)
            if (friday != fridays_list[-1]):
                closest_friday = fridays_list[i + 1]
            else:
                closest_friday = friday
            break

        # print("closest_friday", closest_friday)
        if closest_friday:
            # Filter for the current quote_date and the identified expiry date range
            temp_df = df[
                (df['quote_date'] == quote_date) &
                (df['expire_date'] >= quote_date) &
                (df['expire_date'] <= closest_friday)
            ].copy()
            filtered_df_list.append(temp_df)

    if filtered_df_list:
        return pd.concat(filtered_df_list, ignore_index=True)
    else:
        return pd.DataFrame(columns=df.columns) # Return empty DataFrame if no matches

In [ ]:
# Calculo Aproximado de Griegas (Indicadores Delta y Theta)
from math import exp, sqrt, log
from scipy.stats import norm

def binomial_american_call(S, K, r, q, sigma, T, n_steps=400):
    """
    Precio de opción Call American usando modelo CRR (Cox-Ross-Rubinstein)
    con rendimiento por dividendos continuo q.
    """
    dt = T / n_steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp((r - q) * dt) - d) / (u - d)
    pd = 1 - pu
    disc = np.exp(-r * dt)

    # precios subyacentes al vencimiento
    ST = np.array([S * (u ** j) * (d ** (n_steps - j)) for j in range(n_steps + 1)])

    # payoff en vencimiento (call)
    C = np.maximum(ST - K, 0.0)

    # backward induction con posibilidad de ejercicio anticipado (American)
    for i in range(n_steps - 1, -1, -1):
        # precios en el nivel i
        C = disc * (pu * C[1:] + pd * C[:-1])
        # subyacentes en nivel i
        ST = ST[:i+1] / u  # bajar un nivel
        # valor intrínseco si se ejerce ahora
        intrinsic = np.maximum(ST - K, 0.0)
        # para American: tomar max(valor continuation, intrinsic)
        C = np.maximum(C, intrinsic)
    return float(C)

def greeks_american_call(S, K, r, q, sigma, T, n_steps=400, eps_S=0.01, dt_days=1):
    """
    Calcula Delta y Theta para opción American usando diferencias finitas.
    - eps_S: incremento en precio subyacente (en unidades monetarias).
    - dt_days: cuantos días usar para la aproximación de Theta (1 día por defecto).
    Retorna (price, delta, theta_per_year, theta_per_day).
    """
    price = binomial_american_call(S, K, r, q, sigma, T, n_steps=n_steps)

    # Delta por diferencia central
    C_plus = binomial_american_call(S + eps_S, K, r, q, sigma, T, n_steps=n_steps)
    C_minus = binomial_american_call(S - eps_S, K, r, q, sigma, T, n_steps=n_steps)
    delta = (C_plus - C_minus) / (2 * eps_S)

    # Theta: usar diferencia hacia delante en tiempo (reduciendo T en dt)
    dt = dt_days / 365.0
    if T <= dt:
        # si falta poco tiempo, calcular Theta con dt = T/10 para evitar T<=0
        dt = max(T / 10.0, 1/365.0)
    C_future = binomial_american_call(S, K, r, q, sigma, T - dt, n_steps=n_steps)
    theta_per_year = (price - C_future) / dt  # cambio por unidad de tiempo (años)
    theta_per_day = theta_per_year / 365.0

    return {
        "price": price,
        "delta": delta,
        "theta_per_year": theta_per_year,
        "theta_per_day": theta_per_day
    }

# --- Ejemplo usando tus datos de ejemplo (fila) ---
S = 394.76      # underlying_last
K = 270.0       # strike
# supuestos (debes actualizarlos con valores reales)
r = 0.03        # tasa libre de riesgo anual (3%)
q = 0.014       # rendimiento por dividendos anual continuo (ejemplo 1.4%)
sigma = 0.39    # c_iv (ejemplo tomado de la columna c_iv ~0.39)
T_days = 16.0   # dte de la primera fila de tu screenshot
T = T_days / 365.0

res = greeks_american_call(S, K, r, q, sigma, T, n_steps=800, eps_S=0.1, dt_days=1)
print("Precio (American):", res["price"])
print("Delta:", res["delta"])
print("Theta (año):", res["theta_per_year"])
print("Theta (día):", res["theta_per_day"])

Precio (American): 124.87265367038566
Delta: 0.9993851300176715
Theta (año): 2.5679520398046662
Theta (día): 0.007035485040560729


/tmp/ipython-input-3098936906.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(C)


# EDA

In [ ]:
# Rango de fechas de los contratos
quote_date_min = df_all['quote_date'].min()
quote_date_max = df_all['quote_date'].max()
print(quote_date_min, quote_date_max)

2023-01-03 00:00:00 2023-12-29 00:00:00


In [ ]:
# Rango de fechas de Expiraciones y Strikes por Fechas de Contratos
df_all.groupby('quote_date').agg(
    quote_date_count=('quote_date', 'count'),
    expire_date_min=('expire_date', 'min'),
    expire_date_max=('expire_date', 'max'),
    strike_min=('strike', 'min'),
    strike_max=('strike', 'max'),
    c_last_min=('c_last', 'min'),
    c_last_max=('c_last', 'max')
).head(10)

,quote_date_count,expire_date_min,expire_date_max,strike_min,strike_max,c_last_min,c_last_max
quote_date,,,,,,,
2023-01-03,2880,2023-01-03,2025-12-19,120.0,720.0,0.01,261.90
2023-01-04,2842,2023-01-04,2025-12-19,130.0,720.0,0.01,258.37
2023-01-05,2871,2023-01-05,2025-12-19,120.0,720.0,0.01,261.90
2023-01-06,2886,2023-01-06,2025-12-19,120.0,720.0,0.01,269.17
2023-01-09,2938,2023-01-09,2025-12-19,120.0,720.0,0.01,269.17
2023-01-10,2884,2023-01-10,2025-12-19,120.0,720.0,0.01,269.17
2023-01-11,2949,2023-01-11,2025-12-19,120.0,720.0,0.01,269.17
2023-01-12,3031,2023-01-12,2025-12-19,120.0,720.0,0.01,269.17
2023-01-13,3071,2023-01-13,2025-12-19,120.0,720.0,0.01,269.17


**Nota:** cada contrato con un valor de **Strike** y **Fecha de Expiracion** diferente.

In [ ]:
df_all.shape

(757848, 19)

In [ ]:
fridays_list = get_fridays_in_range(quote_date_min, quote_date_max)
print(fridays_list)

[Timestamp('2023-01-06 00:00:00'), Timestamp('2023-01-13 00:00:00'), Timestamp('2023-01-20 00:00:00'), Timestamp('2023-01-27 00:00:00'), Timestamp('2023-02-03 00:00:00'), Timestamp('2023-02-10 00:00:00'), Timestamp('2023-02-17 00:00:00'), Timestamp('2023-02-24 00:00:00'), Timestamp('2023-03-03 00:00:00'), Timestamp('2023-03-10 00:00:00'), Timestamp('2023-03-17 00:00:00'), Timestamp('2023-03-24 00:00:00'), Timestamp('2023-03-31 00:00:00'), Timestamp('2023-04-07 00:00:00'), Timestamp('2023-04-14 00:00:00'), Timestamp('2023-04-21 00:00:00'), Timestamp('2023-04-28 00:00:00'), Timestamp('2023-05-05 00:00:00'), Timestamp('2023-05-12 00:00:00'), Timestamp('2023-05-19 00:00:00'), Timestamp('2023-05-26 00:00:00'), Timestamp('2023-06-02 00:00:00'), Timestamp('2023-06-09 00:00:00'), Timestamp('2023-06-16 00:00:00'), Timestamp('2023-06-23 00:00:00'), Timestamp('2023-06-30 00:00:00'), Timestamp('2023-07-07 00:00:00'), Timestamp('2023-07-14 00:00:00'), Timestamp('2023-07-21 00:00:00'), Timestamp('20

In [ ]:
df_filtered = filter_by_closest_friday_expiry(df_all, fridays_list)

In [ ]:
# df_all_tmp = (df_all[
#       (df_all['quote_date'].isin(['2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06']))
#     ]
# )

In [ ]:
# df_all_tmp['quote_date'].value_counts()

In [ ]:
#Validacion: Dataframe original
(df_all[
      (df_all['quote_date'] >= '2022-01-03') &
      (df_all['quote_date'] <= '2022-01-07')
    ].shape
)

(0, 19)

In [ ]:
#Validacion: Dataframe filtered
(df_filtered[
      (df_filtered['quote_date'] >= '2022-01-03') &
      (df_filtered['quote_date'] <= '2022-01-07')
    ].shape
)

(0, 19)

In [ ]:
df_filtered.shape

(99100, 19)

In [ ]:
# Rango de fechas de los contratos
quote_date_min = df_filtered['quote_date'].min()
quote_date_max = df_filtered['quote_date'].max()
print(quote_date_min, quote_date_max)

2023-01-03 00:00:00 2023-12-29 00:00:00


In [ ]:
#Aprox. deberia ser 250 dias aprox. de ejecucion de la bolsa por año
df_filtered['quote_date'].unique()

<DatetimeArray>
['2023-04-03 00:00:00', '2023-04-04 00:00:00', '2023-04-05 00:00:00',
 '2023-04-06 00:00:00', '2023-04-10 00:00:00', '2023-04-11 00:00:00',
 '2023-04-12 00:00:00', '2023-04-13 00:00:00', '2023-04-14 00:00:00',
 '2023-04-17 00:00:00',
 ...
 '2023-12-15 00:00:00', '2023-12-18 00:00:00', '2023-12-19 00:00:00',
 '2023-12-20 00:00:00', '2023-12-21 00:00:00', '2023-12-22 00:00:00',
 '2023-12-26 00:00:00', '2023-12-27 00:00:00', '2023-12-28 00:00:00',
 '2023-12-29 00:00:00']
Length: 250, dtype: datetime64[ns]

In [ ]:
# % de reduccion de datos
((df_all.shape[0] - df_filtered.shape[0]) / df_all.shape[0]) * 100

86.9234991713378

In [ ]:
# Promedio de contratos por dia (cada contrato con un valor de Strike diferente)
df_all.groupby('quote_date')['quote_date'].count().mean()

np.float64(3031.392)

In [ ]:
# Una Fecha de Contrato con un mismo Strike y Diferentes fechas de Expiraciones
(df_all[
      (df_all['quote_date'] == '2023-03-01') &
      (df_all['strike'] == 270)
    ]
 .sort_values(by='expire_date')
).head(5)

,quote_date,expire_date,dte,underlying_last,strike,strike_distance,strike_distance_pct,c_last,c_bid,c_ask,c_iv,c_volume,c_delta,c_gamma,c_vega,c_theta,daysToExp,c_mid_price,moneyness_category
558100,2023-03-01,2023-03-17,15.96,394.76,270.0,124.8,0.316,148.00,125.14,125.36,0.93975,0.0,0.97048,0.00051,0.04558,-0.11844,16,125.250,ITM
558410,2023-03-01,2023-03-31,29.96,394.76,270.0,124.8,0.316,129.77,125.13,125.39,0.70506,0.0,0.96937,0.00049,0.06350,-0.06773,30,125.260,ITM
558822,2023-03-01,2023-05-19,78.96,394.76,270.0,124.8,0.316,135.26,126.19,126.70,0.38276,0.0,0.98385,0.00044,0.05806,-0.03012,79,126.445,ITM
558960,2023-03-01,2023-06-16,106.96,394.76,270.0,124.8,0.316,132.15,127.13,128.25,0.38934,0.0,0.97019,0.00075,0.12781,-0.04096,107,127.690,ITM
559140,2023-03-01,2023-06-30,120.96,394.76,270.0,124.8,0.316,120.59,127.20,128.46,0.38115,0.0,0.96453,0.00075,0.15361,-0.03913,121,127.830,ITM


#Generar CSV

In [ ]:
df_filtered.shape

(99100, 19)

In [ ]:
df_filtered.columns

Index(['quote_date', 'expire_date', 'dte', 'underlying_last', 'strike',
       'strike_distance', 'strike_distance_pct', 'c_last', 'c_bid', 'c_ask',
       'c_iv', 'c_volume', 'c_delta', 'c_gamma', 'c_vega', 'c_theta',
       'daysToExp', 'c_mid_price', 'moneyness_category'],
      dtype='object')

In [ ]:
output_name = 'spy_options_call_output_' + anio + '.csv'
df_filtered.to_csv(output_name, index=False)
print(f"se guardo el archivo CSV para el año {anio}")

se guardo el archivo CSV para el año 2023
